In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# settings
seed = 1
batch_size = 800
test_batch_size = 300
lr = 0.01
momentum = 0.9
weight_decay = 5e-4
epochs = 200
log_interval = 16
save_model = False
criterion = nn.CrossEntropyLoss()
torch.backends.cudnn.benchmark=True

In [3]:
# ResNet inner blocks
class planeBlock(nn.Module):
    def __init__(self, inplane, outplane, stride=1, padding=1, downsample=False):
        super(planeBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplane, outplane, kernel_size=3, stride=stride, padding=padding, bias=True)
        self.bn1 = nn.BatchNorm2d(inplane)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout2d(0.5)
        self.conv2 =  nn.Conv2d(outplane, outplane,  kernel_size=3, stride=1, padding=padding, bias=True)
        self.bn2 = nn.BatchNorm2d(outplane)

        # for change identity size
        if stride != 1 or inplane != outplane :
            self.downsample = nn.Sequential(
            nn.Conv2d(inplane, outplane, kernel_size=1, stride=stride, bias=True),
            nn.BatchNorm2d(outplane),
            )
        else :
            self.downsample = downsample

        print(inplane, outplane,)
        
    def forward(self, x):
        identity = x        # inputs shorcut

        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)

        if self.downsample is not False :
            identity = self.downsample(x)
            #print(self.stride)

        out += identity        
#        out = self.relu(out)

        return out

In [4]:
class resNet(nn.Module):
    def __init__(self):
        super(resNet, self).__init__()
#        self.inplanes = 128

        # top convolution 
        self.base = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1,),
                nn.Dropout2d(0.3),
        )
        
        
        # inner blocks settings
        layers =  [
                planeBlock(64, 64),
                planeBlock(64, 64),
                planeBlock(64, 64),
                planeBlock(64, 128, stride=2,),
                planeBlock(128, 128, stride=1,),
                planeBlock(128, 128, stride=1,),
                planeBlock(128, 128, stride=1,),
                planeBlock(128, 128, stride=1,),
                planeBlock(128, 256, stride=2,),
                planeBlock(256, 256, stride=1,),
                planeBlock(256, 256, stride=1,),
                planeBlock(256, 256, stride=1,),
                planeBlock(256, 256, stride=1,),
                planeBlock(256, 256, stride=1,),
                planeBlock(256, 512, stride=2,),
                planeBlock(512, 512, stride=1,),
                planeBlock(512, 512, stride=1,),
        ]
        
        self.planes = nn.Sequential(*layers)
        
        
        # pooling layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        
        # output side
        self.classifier = nn.Sequential(
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, num_classes),
        )
        

    def forward(self, x):
        x = self.base(x)
        x = self.planes(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [5]:
def train(model, device, train_loader, optimizer, log_interval, epoch, epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # proccessing display for training
        if batch_idx % log_interval == 0:
            print('Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, epochs, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())) 


In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).sum().item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    # proccessing display for test   
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
# use device to Cuda or CPU
use_cuda =  torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}

torch.manual_seed(seed)


# get datasets
train_dataset=datasets.CIFAR100('./data_c100', train=True, download=True,
                                transform=transforms.Compose([
                                        transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                       
                                        ])
                               )

test_dataset=datasets.CIFAR100('./data_c100', train=False, download=True,
                                transform=transforms.Compose([
                                        transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                        ])
                              )


# classes count
labels = [i[1] for i in train_dataset]
num_classes = max(labels) + 1

# show datasets shape
image = train_dataset[0]
print(image[0].size())
print('Labels:', num_classes)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([3, 32, 32])
Labels: 100


In [8]:
# datasets to DataLoader
train_loader = torch.utils.data.DataLoader( dataset=train_dataset, 
                                            batch_size=batch_size, shuffle=True, 
                                            **kwargs,
                                          )

test_loader = torch.utils.data.DataLoader( dataset=test_dataset, 
                                            batch_size=test_batch_size, shuffle=False,
                                            **kwargs
                                         )

# check tensor shape
for images, labels in train_loader:
    print(images.size())
    print(images[0].size())
    print(labels.size())
    
    break

torch.Size([800, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([800])


In [9]:
# build model
model = resNet().to(device)
optimizer = optim.SGD(model.parameters(),
                       lr=lr,
                       momentum=momentum,
                       weight_decay=weight_decay,
                      )

# model run
def run():
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, log_interval, epoch, epochs)
        test(model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"resnet.pt")

64 64
64 64
64 64
64 128
128 128
128 128
128 128
128 128
128 256
256 256
256 256
256 256
256 256
256 256
256 512
512 512
512 512


In [10]:
%%time
print('Use device is', str(device).upper())

run()

Use device is CUDA
Train Epoch: 1/200 [0/50000 (0%)]	Loss: 4.637008
Train Epoch: 1/200 [12800/50000 (25%)]	Loss: 4.595077
Train Epoch: 1/200 [25600/50000 (51%)]	Loss: 4.523753
Train Epoch: 1/200 [38400/50000 (76%)]	Loss: 4.479592

Test set: Average loss: 0.0150, Accuracy: 463/10000 (5%)

Train Epoch: 2/200 [0/50000 (0%)]	Loss: 4.432254
Train Epoch: 2/200 [12800/50000 (25%)]	Loss: 4.318320
Train Epoch: 2/200 [25600/50000 (51%)]	Loss: 4.310691
Train Epoch: 2/200 [38400/50000 (76%)]	Loss: 4.270717

Test set: Average loss: 0.0143, Accuracy: 778/10000 (8%)

Train Epoch: 3/200 [0/50000 (0%)]	Loss: 4.218570
Train Epoch: 3/200 [12800/50000 (25%)]	Loss: 4.171470
Train Epoch: 3/200 [25600/50000 (51%)]	Loss: 4.131759
Train Epoch: 3/200 [38400/50000 (76%)]	Loss: 4.108551

Test set: Average loss: 0.0136, Accuracy: 1078/10000 (11%)

Train Epoch: 4/200 [0/50000 (0%)]	Loss: 4.064671
Train Epoch: 4/200 [12800/50000 (25%)]	Loss: 4.022525
Train Epoch: 4/200 [25600/50000 (51%)]	Loss: 4.019542
Train Epoch:


Test set: Average loss: 0.0074, Accuracy: 4283/10000 (43%)

Train Epoch: 31/200 [0/50000 (0%)]	Loss: 2.150987
Train Epoch: 31/200 [12800/50000 (25%)]	Loss: 2.038830
Train Epoch: 31/200 [25600/50000 (51%)]	Loss: 2.098625
Train Epoch: 31/200 [38400/50000 (76%)]	Loss: 2.059358

Test set: Average loss: 0.0075, Accuracy: 4280/10000 (43%)

Train Epoch: 32/200 [0/50000 (0%)]	Loss: 1.993166
Train Epoch: 32/200 [12800/50000 (25%)]	Loss: 2.042514
Train Epoch: 32/200 [25600/50000 (51%)]	Loss: 1.922102
Train Epoch: 32/200 [38400/50000 (76%)]	Loss: 2.013554

Test set: Average loss: 0.0072, Accuracy: 4398/10000 (44%)

Train Epoch: 33/200 [0/50000 (0%)]	Loss: 1.985626
Train Epoch: 33/200 [12800/50000 (25%)]	Loss: 1.972549
Train Epoch: 33/200 [25600/50000 (51%)]	Loss: 1.961795
Train Epoch: 33/200 [38400/50000 (76%)]	Loss: 1.939509

Test set: Average loss: 0.0073, Accuracy: 4357/10000 (44%)

Train Epoch: 34/200 [0/50000 (0%)]	Loss: 1.964078
Train Epoch: 34/200 [12800/50000 (25%)]	Loss: 1.958552
Train 

Train Epoch: 60/200 [38400/50000 (76%)]	Loss: 1.014484

Test set: Average loss: 0.0073, Accuracy: 4838/10000 (48%)

Train Epoch: 61/200 [0/50000 (0%)]	Loss: 0.801430
Train Epoch: 61/200 [12800/50000 (25%)]	Loss: 0.904869
Train Epoch: 61/200 [25600/50000 (51%)]	Loss: 0.818156
Train Epoch: 61/200 [38400/50000 (76%)]	Loss: 0.928955

Test set: Average loss: 0.0075, Accuracy: 4829/10000 (48%)

Train Epoch: 62/200 [0/50000 (0%)]	Loss: 0.797035
Train Epoch: 62/200 [12800/50000 (25%)]	Loss: 0.865327
Train Epoch: 62/200 [25600/50000 (51%)]	Loss: 0.900730
Train Epoch: 62/200 [38400/50000 (76%)]	Loss: 0.867969

Test set: Average loss: 0.0075, Accuracy: 4780/10000 (48%)

Train Epoch: 63/200 [0/50000 (0%)]	Loss: 0.868281
Train Epoch: 63/200 [12800/50000 (25%)]	Loss: 0.878411
Train Epoch: 63/200 [25600/50000 (51%)]	Loss: 0.846105
Train Epoch: 63/200 [38400/50000 (76%)]	Loss: 0.956051

Test set: Average loss: 0.0077, Accuracy: 4819/10000 (48%)

Train Epoch: 64/200 [0/50000 (0%)]	Loss: 0.761279
Train 

Train Epoch: 90/200 [25600/50000 (51%)]	Loss: 0.358422
Train Epoch: 90/200 [38400/50000 (76%)]	Loss: 0.325560

Test set: Average loss: 0.0086, Accuracy: 4866/10000 (49%)

Train Epoch: 91/200 [0/50000 (0%)]	Loss: 0.264207
Train Epoch: 91/200 [12800/50000 (25%)]	Loss: 0.305408
Train Epoch: 91/200 [25600/50000 (51%)]	Loss: 0.344529
Train Epoch: 91/200 [38400/50000 (76%)]	Loss: 0.331524

Test set: Average loss: 0.0087, Accuracy: 4847/10000 (48%)

Train Epoch: 92/200 [0/50000 (0%)]	Loss: 0.280008
Train Epoch: 92/200 [12800/50000 (25%)]	Loss: 0.286903
Train Epoch: 92/200 [25600/50000 (51%)]	Loss: 0.287016
Train Epoch: 92/200 [38400/50000 (76%)]	Loss: 0.342497

Test set: Average loss: 0.0088, Accuracy: 4843/10000 (48%)

Train Epoch: 93/200 [0/50000 (0%)]	Loss: 0.295143
Train Epoch: 93/200 [12800/50000 (25%)]	Loss: 0.315234
Train Epoch: 93/200 [25600/50000 (51%)]	Loss: 0.291802
Train Epoch: 93/200 [38400/50000 (76%)]	Loss: 0.289620

Test set: Average loss: 0.0087, Accuracy: 4827/10000 (48%)

T


Test set: Average loss: 0.0094, Accuracy: 4850/10000 (48%)

Train Epoch: 120/200 [0/50000 (0%)]	Loss: 0.110363
Train Epoch: 120/200 [12800/50000 (25%)]	Loss: 0.131729
Train Epoch: 120/200 [25600/50000 (51%)]	Loss: 0.145272
Train Epoch: 120/200 [38400/50000 (76%)]	Loss: 0.132703

Test set: Average loss: 0.0094, Accuracy: 4812/10000 (48%)

Train Epoch: 121/200 [0/50000 (0%)]	Loss: 0.145554
Train Epoch: 121/200 [12800/50000 (25%)]	Loss: 0.127661
Train Epoch: 121/200 [25600/50000 (51%)]	Loss: 0.146856
Train Epoch: 121/200 [38400/50000 (76%)]	Loss: 0.160701

Test set: Average loss: 0.0093, Accuracy: 4912/10000 (49%)

Train Epoch: 122/200 [0/50000 (0%)]	Loss: 0.126019
Train Epoch: 122/200 [12800/50000 (25%)]	Loss: 0.131345
Train Epoch: 122/200 [25600/50000 (51%)]	Loss: 0.133653
Train Epoch: 122/200 [38400/50000 (76%)]	Loss: 0.135502

Test set: Average loss: 0.0097, Accuracy: 4777/10000 (48%)

Train Epoch: 123/200 [0/50000 (0%)]	Loss: 0.122211
Train Epoch: 123/200 [12800/50000 (25%)]	Loss: 0

Train Epoch: 149/200 [12800/50000 (25%)]	Loss: 0.079661
Train Epoch: 149/200 [25600/50000 (51%)]	Loss: 0.079691
Train Epoch: 149/200 [38400/50000 (76%)]	Loss: 0.089338

Test set: Average loss: 0.0093, Accuracy: 4950/10000 (50%)

Train Epoch: 150/200 [0/50000 (0%)]	Loss: 0.080914
Train Epoch: 150/200 [12800/50000 (25%)]	Loss: 0.094138
Train Epoch: 150/200 [25600/50000 (51%)]	Loss: 0.097470
Train Epoch: 150/200 [38400/50000 (76%)]	Loss: 0.112429

Test set: Average loss: 0.0095, Accuracy: 4892/10000 (49%)

Train Epoch: 151/200 [0/50000 (0%)]	Loss: 0.083067
Train Epoch: 151/200 [12800/50000 (25%)]	Loss: 0.068146
Train Epoch: 151/200 [25600/50000 (51%)]	Loss: 0.087233
Train Epoch: 151/200 [38400/50000 (76%)]	Loss: 0.105515

Test set: Average loss: 0.0095, Accuracy: 4901/10000 (49%)

Train Epoch: 152/200 [0/50000 (0%)]	Loss: 0.088146
Train Epoch: 152/200 [12800/50000 (25%)]	Loss: 0.076401
Train Epoch: 152/200 [25600/50000 (51%)]	Loss: 0.075647
Train Epoch: 152/200 [38400/50000 (76%)]	Loss: 0

Train Epoch: 178/200 [38400/50000 (76%)]	Loss: 0.081788

Test set: Average loss: 0.0095, Accuracy: 4921/10000 (49%)

Train Epoch: 179/200 [0/50000 (0%)]	Loss: 0.075833
Train Epoch: 179/200 [12800/50000 (25%)]	Loss: 0.055321
Train Epoch: 179/200 [25600/50000 (51%)]	Loss: 0.066813
Train Epoch: 179/200 [38400/50000 (76%)]	Loss: 0.066533

Test set: Average loss: 0.0094, Accuracy: 4954/10000 (50%)

Train Epoch: 180/200 [0/50000 (0%)]	Loss: 0.054740
Train Epoch: 180/200 [12800/50000 (25%)]	Loss: 0.050376
Train Epoch: 180/200 [25600/50000 (51%)]	Loss: 0.054206
Train Epoch: 180/200 [38400/50000 (76%)]	Loss: 0.068979

Test set: Average loss: 0.0095, Accuracy: 4870/10000 (49%)

Train Epoch: 181/200 [0/50000 (0%)]	Loss: 0.068316
Train Epoch: 181/200 [12800/50000 (25%)]	Loss: 0.073300
Train Epoch: 181/200 [25600/50000 (51%)]	Loss: 0.050645
Train Epoch: 181/200 [38400/50000 (76%)]	Loss: 0.067337

Test set: Average loss: 0.0095, Accuracy: 4927/10000 (49%)

Train Epoch: 182/200 [0/50000 (0%)]	Loss: 0